# 2.データ準備と前処理


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードの「データ準備と前処理」部分について、ソクラテス式チャットボットの文脈で説明していきます。

### 1. データの検証（validate_message_format関数）
```python
def validate_message_format(message):
```
この関数は、会話データが正しい形式になっているかをチェックします。

例えば以下のような会話データが正しいフォーマットとして扱われます：
```json
{
    "role": "user",
    "content": "AIについてどう思いますか？"
}
{
    "role": "model",
    "content": "AIについて、まず何が知りたいのでしょうか？私たちで一緒に考えていきましょう。"
}
```

### 2. データセットの準備（prepare_dataset関数）
```python
def prepare_dataset():
```
この関数は主に3つのことを行います：

1. **会話データの読み込みと検証**:
   - JSONファイルから会話データを読み込みます
   - ユーザーとモデルの会話が交互になっているか確認します

2. **会話の整形**:
   - Gemmaモデル用の特別な形式に会話を変換します
   - 例：
     ```
     ユーザー: AIの倫理について考えてみましょう
     モデル: その問題について、まず何が気になっているのでしょうか？
     ```

3. **長さのチェック**:
   - 会話が長すぎないかチェックします（MAX_SEQUENCE_LENGTH以内）

### 3. トークン化（tokenize_function）
```python
def tokenize_function(examples):
```
会話テキストを機械学習モデルが理解できる数値（トークン）に変換します。

例：
```
"なぜでしょうか？" → [1234, 5678, 9012]
```

### 4. 前処理（preprocess_function）
```python
def preprocess_function(examples):
```
ソクラテス式の特徴的な表現に注目するよう設定します：

1. **文末パターンの強調**:
   - "だろうか"、"ではないか"などの問いかけ
   - "なるほど"、"興味深い"などの応答

2. **接続詞の強調**:
   - "しかし"、"だから"などの論理展開

3. **句読点の強調**:
   - 「、」「。」「！」「？」の位置を記録

これにより、モデルがソクラテス式の対話の特徴（問いかけ、論理的な展開、適切な間の取り方など）により注意を払うようになります。

このような前処理によって、モデルは：
- 適切なタイミングで質問を投げかけ
- 論理的な会話の流れを作り
- ソクラテス式の対話らしい応答を生成
できるように学習データが整えられます。

        
</div>
    </details>
</div>


In [ ]:

def validate_message_format(message):
    """Validate message format"""
    if not isinstance(message, dict):
        return False
    if 'role' not in message or 'content' not in message:
        return False
    if message['role'] not in ['user', 'model']:
        return False
    if not isinstance(message['content'], str):
        return False
    return True







<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

この関数は対話データの各メッセージが正しい形式かチェックします：
- 辞書型であること
- 'role'と'content'のキーがあること
- roleが'user'か'model'であること
- contentが文字列であること
        
</div>
    </details>
</div>


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #ffe6e6; padding: 10px; border-left: 5px solid #8b0000; margin: 10px; width: 95%;">
    <summary style="color: #8b0000; font-size: 1.17em; font-weight: bold;">補足</summary>
    <div style="color: #8b0000;">


対話がuserからはじまるかどうかはチェックしてない？


</div>
</div>

In [ ]:
def prepare_dataset():
    conversations = []
    
    try:
        with open(DIALOGUE_JSON_PATH, 'r', encoding='utf-8') as f:
            dialogue_data = json.load(f)
            
        for dialogue in dialogue_data:
            messages = dialogue.get('messages', [])
            
            # Validate message format
            if not all(validate_message_format(msg) for msg in messages):
                logging.warning(f"Skipped dialogue due to invalid message format")
                continue
                
            # Build conversation checking user->model sequence
            current_conversation = []
            valid_sequence = True
            
            for i in range(0, len(messages)-1, 2):
                if (i+1 < len(messages) and 
                    messages[i]['role'] == 'user' and 
                    messages[i+1]['role'] == 'model'):
                    current_conversation.extend([messages[i], messages[i+1]])
                else:
                    valid_sequence = False
                    break
            
            # 会話が有効で内容があれば処理
            if valid_sequence and current_conversation:
                # Gemmaモデル用のチャットテンプレートを適用
                formatted_text = tokenizer.apply_chat_template(
                    current_conversation,
                    tokenize=False,
                    add_generation_prompt=True
                )
                
                # トークン数をチェック
                tokens = tokenizer.encode(formatted_text)
                if len(tokens) <= MAX_SEQUENCE_LENGTH:
                    # 長さが適切なら追加
                    conversations.append({"text": formatted_text})
                else:
                    # 長すぎる場合はスキップ
                    logging.warning(f"Skipped conversation due to length: {len(tokens)} tokens")
            
    except Exception as e:
        # エラーが発生した場合はログに記録して再発生
        logging.error(f"Error processing dialogue file: {str(e)}")
        raise
    
    # 有効な会話がなければエラー
    if not conversations:
        raise ValueError("No valid conversations found in the dialogue file")

    # 処理した会話数をログに記録    
    logging.info(f"Processed {len(conversations)} valid conversations")

    # 会話リストからデータセットを作成して返す
    return Dataset.from_list(conversations)




<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">


**この関数が行っていること**:
1. JSONファイルから対話データを読み込む
2. 各対話について：
   - メッセージ形式を検証
   - ユーザー→モデルの順序を確認
   - Gemmaモデル用に形式を整える
   - トークン数をチェック
3. 有効な会話のみを保持
4. データセットとして返す

**重要なチェック項目**:
- メッセージの形式
- 会話の順序（ユーザー→モデル）
- トークンの長さ制限
- エラー処理とログ記録

このコードは、生の対話データを機械学習モデルで使用できる形式に変換する重要な前処理ステップを担っています。



        
</div>
    </details>
</div>


In [ ]:

# Dataset preparation
dataset = prepare_dataset()









<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">


        
**データセットの準備**:

- 先ほど説明した`prepare_dataset()`関数を使用してデータセットを作成
- 対話データをモデル学習用のフォーマットに変換
</div>
    </details>
</div>


In [ ]:
# Check dataset structure
print("Dataset structure:")
print(dataset[0])  # Display first element
print("\nDataset features:")
print(dataset.features)








<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

**データセット構造の確認**:

- データセットの最初の要素を表示して構造を確認
- 例えば以下のような出力が期待される：
```
Dataset structure:
{'text': 'User: こんにちは\nAssistant: はい、こんにちは。...'}

Dataset features:
{'text': Value(dtype='string', id=None)}
```
        
</div>
    </details>
</div>


In [ ]:
# Optimize dataset batch processing
dataset = dataset.select(range(len(dataset))).shuffle(seed=42)







<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">


**データセットの最適化**:
```python
# データセットのバッチ処理の最適化
dataset = dataset.select(range(len(dataset))).shuffle(seed=42)
```
- `select(range(len(dataset)))`: データセット全体を選択
- `shuffle(seed=42)`: データをランダムに並び替え
  - `seed=42`: 再現性のために固定の乱数シードを使用
  - 学習時のデータの偏りを防ぐため

        
</div>
    </details>
</div>


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>


<div style="background-color: #e6f3ff; padding: 10px; border-left: 5px solid #2196F3; margin: 10px; width: 95%;">
<summary style="color: #2196F3; font-size: 1.17em; font-weight: bold;">Q. 質問内容</summary>
<p style="color: #2196F3;">

jsonの中に例えば

user1発話目
model１発話目
user2発話目
model2発話目

の計４発話がひとくくりになって、１つの対話になっていて、その対話が１００個あった場合、シャッフルされるのはその１００個の対話の配置ですか？それとも
”user1発話目とmodel１発話目の１ペア”を１つの単位として切り出して、１ペアを最小単位としてシャッフルされるんですか

</p>
</div>


<div style="background-color: #f5f5f5; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #4CAF50; font-size: 1.17em; font-weight: bold;">A. 解答</summary>
        <div style="color: #4CAF50;">





コードを確認すると、シャッフルされるのは「対話単位」です。つまり、100個の対話の配置がシャッフルされます。

これは以下のコードの流れから分かります：

1. **対話の処理（`prepare_dataset`関数内）**:
```python
for dialogue in dialogue_data:
    messages = dialogue.get('messages', [])
    
    # ... 検証処理 ...
    
    # user->model のペアを保持したまま会話を構築
    current_conversation = []
    for i in range(0, len(messages)-1, 2):
        if (i+1 < len(messages) and 
            messages[i]['role'] == 'user' and 
            messages[i+1]['role'] == 'model'):
            current_conversation.extend([messages[i], messages[i+1]])
    
    # 会話全体を1つのユニットとして保存
    if valid_sequence and current_conversation:
        formatted_text = tokenizer.apply_chat_template(
            current_conversation,  # 会話全体を1つのテキストに
            tokenize=False,
            add_generation_prompt=True
        )
        conversations.append({"text": formatted_text})
```

2. **シャッフル処理**:
```python
dataset = dataset.select(range(len(dataset))).shuffle(seed=42)
```

つまり：
```
[対話1]
- user1発話目
- model1発話目
- user2発話目
- model2発話目

[対話2]
- user1発話目
- model1発話目
- user2発話目
- model2発話目

... （100個の対話）
```

これらの対話単位でシャッフルされます。対話内の発話順序（user→model→user→model）は維持されます。

シャッフル後のイメージ：
```
[対話54]
- user1発話目
- model1発話目
- user2発話目
- model2発話目

[対話12]
- user1発話目
- model1発話目
- user2発話目
- model2発話目

[対話87]
...
```

個々の発話ペアをバラバラにシャッフルしないのは、文脈の一貫性を保つためです。対話の流れを保持したまま、異なる対話のトピックが適度に混ざるようにしています。

        
</div>
    </details>
</div>

In [ ]:

# Log dataset size
logging.info(f"Total dataset size: {len(dataset)}")
log_memory_usage()





<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">


 **サイズとメモリ使用量の記録**:
```python
# データセットサイズのログ記録
logging.info(f"Total dataset size: {len(dataset)}")
log_memory_usage()
```
- データセットの総サイズをログに記録
- メモリ使用状況を確認（CPU・GPUのメモリ使用量）

        
</div>
    </details>
</div>


In [ ]:

def tokenize_function(examples):
    result = tokenizer(
        examples['text'],
        truncation=True,
        max_length=TOKENIZE_MAX_LENGTH,      # 256 から TOKENIZE_MAX_LENGTH に変更
        padding='max_length',
        add_special_tokens=True,
        return_tensors=None,
    )
    return result


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">







この`tokenize_function`は、テキストをモデルが理解できる数値形式（トークン）に変換する関数です。詳しく説明します：

```python
def tokenize_function(examples):
    # examplesは複数の対話テキストを含む辞書
    # examples['text']の例：
    # ["User: こんにちは\nAssistant: はい、こんにちは。", 
    #  "User: 天気は？\nAssistant: 今日は晴れです。", ...]
    
    result = tokenizer(
        examples['text'],
        truncation=True,                    # 長すぎるテキストを切り詰める
        max_length=TOKENIZE_MAX_LENGTH,     # 最大長（例：512）
        padding='max_length',               # 短いテキストを最大長まで埋める
        add_special_tokens=True,            # [BOS], [EOS]などの特殊トークンを追加
        return_tensors=None,                # 通常のPythonリストとして返す
    )
    return result
```

**具体例**:
```python
# 入力テキスト
text = "User: こんにちは\nAssistant: はい、こんにちは。"

# トークン化の結果（イメージ）
result = {
    'input_ids': [1, 245, 567, 89, ..., 0, 0],  # テキストを数値に変換
    'attention_mask': [1, 1, 1, 1, ..., 0, 0]   # 実際のトークンは1、パディングは0
}
```

**各パラメータの説明**:

1. **truncation=True**:
   ```python
   # 長すぎるテキスト
   長いテキスト = "とても長い文章..." # 1000トークン
   
   # 切り詰め後
   結果 = "とても長い..." # TOKENIZE_MAX_LENGTH（例：512）トークンまで
   ```

2. **padding='max_length'**:
   ```python
   # 短いテキスト
   短いテキスト = "こんにちは"  # 5トークン
   
   # パディング後
   結果 = "こんにちは[PAD][PAD]..."  # TOKENIZE_MAX_LENGTHまで[PAD]で埋める
   ```

3. **add_special_tokens=True**:
   ```python
   # 特殊トークンの追加
   元テキスト = "こんにちは"
   結果 = "[BOS]こんにちは[EOS]"  # 文の始まりと終わりを示すトークンを追加
   ```

**使用例**:
```python
# データセットに適用
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,  # バッチ処理で効率化
    num_proc=2,    # 複数プロセスで並列処理
    desc="Tokenizing datasets"
)
```

このトークン化は、テキストをモデルが処理できる形式に変換する重要な前処理ステップです。すべての入力テキストを同じ長さに揃え、特殊トークンを適切に配置することで、モデルの学習が効率的に行えるようになります。

        
</div>
    </details>
</div>


In [ ]:

# Add dataset preprocessing
def preprocess_function(examples):
    # Pattern definitions
    end_patterns = [
        "だろうか", "ではないか", "のではないか", "かね",
        "なるほど", "興味深い", "考えてみよう"
    ]
    
    # Conjunction patterns
    conjunctions = [
        "しかし", "だから", "それでは", "すなわち",
        "たとえば", "つまり", "ならば", "もし"
    ]
    
    # Get tokenized texts
    texts = tokenizer.batch_decode(examples['input_ids'])
    new_attention_masks = []
    
    for text, mask in zip(texts, examples['attention_mask']):
        if not isinstance(mask, list):
            mask = mask.tolist()
        
        # Create new attention mask (base value 0.8)
        new_mask = [0.8] * len(mask)
        
        # Split into sentences
        sentences = text.split('。')
        current_pos = 0
        
        for sentence in sentences:
            if not sentence.strip():
                continue
                
            # Detect and emphasize end patterns
            for pattern in end_patterns:
                if pattern in sentence:
                    # Locate pattern position
                    pattern_tokens = tokenizer.encode(pattern, add_special_tokens=False)
                    pattern_len = len(pattern_tokens)
                    
                    # Emphasize tokens containing pattern and surrounding tokens
                    pattern_start = current_pos + len(tokenizer.encode(sentence, add_special_tokens=False)) - pattern_len
                    for i in range(max(0, pattern_start - 2), min(len(mask), pattern_start + pattern_len + 2)):
                        new_mask[i] = 1.0  # Maximum attention for pattern parts
            
            # Detect and emphasize conjunctions
            for conj in conjunctions:
                if conj in sentence:
                    # Locate conjunction position
                    conj_tokens = tokenizer.encode(conj, add_special_tokens=False)
                    conj_pos = len(tokenizer.encode(sentence.split(conj)[0], add_special_tokens=False))
                    
                    # Emphasize tokens before and after conjunction (slightly lower)
                    for i in range(max(0, current_pos + conj_pos - 1), 
                                 min(len(mask), current_pos + conj_pos + len(conj_tokens) + 1)):
                        new_mask[i] = 0.9
            
            # Emphasize punctuation marks
            for i, char in enumerate(sentence):
                if char in '、。！？':
                    # Locate punctuation position
                    punct_pos = len(tokenizer.encode(sentence[:i], add_special_tokens=False))
                    # Emphasize tokens around punctuation
                    for j in range(max(0, current_pos + punct_pos - 1),
                                 min(len(mask), current_pos + punct_pos + 2)):
                        new_mask[j] = 0.95
            
            # Update position for next sentence
            current_pos += len(tokenizer.encode(sentence + '。', add_special_tokens=False))
        
        # Set special token masks to 1.0
        if tokenizer.bos_token_id is not None:
            new_mask[0] = 1.0  # BOS token
        if tokenizer.eos_token_id is not None:
            new_mask[-1] = 1.0  # EOS token
            
        new_attention_masks.append(new_mask)

    examples['attention_mask'] = new_attention_masks
    return examples


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">







このコードは、ソクラテス式問答のテキストに対して、重要な部分により注意を向けるように設定する前処理関数です。具体例を交えて説明します：

1. **パターンの定義**:
```python
# 文末パターン
end_patterns = [
    "だろうか", "ではないか", "のではないか", "かね",
    "なるほど", "興味深い", "考えてみよう"
]
# 例：「真理とは何だろうか」「それは興味深い考えではないか」

# 接続パターン
conjunctions = [
    "しかし", "だから", "それでは", "すなわち",
    "たとえば", "つまり", "ならば", "もし"
]
# 例：「しかし、それは本当の知恵といえるでしょうか」
```

2. **具体的な処理例**:
```python
# 入力テキストの例
text = """
User: 知識と知恵の違いについて考えてみましょう。
Assistant: なるほど、興味深いテーマですね。まず、知識とは何だろうか。たとえば、本から得られる情報は知識といえます。しかし、その知識を適切に活用できることが、すなわち知恵ではないでしょうか。
"""
```

3. **注意度の設定**:
```python
# 基本の注意度: 0.8
"知識と知恵の違いについて" -> [0.8, 0.8, 0.8, ...]

# 文末パターンの強調（注意度: 1.0）
"何だろうか" -> [1.0, 1.0, 1.0]
"ではないでしょうか" -> [1.0, 1.0, 1.0]

# 接続詞の強調（注意度: 0.9）
"しかし" -> [0.9, 0.9]
"すなわち" -> [0.9, 0.9, 0.9]

# 句読点の強調（注意度: 0.95）
"、" -> [0.95]
"。" -> [0.95]
```

4. **処理の流れ**:
```python
# 例文：「知識とは何だろうか。しかし、それは本当の答えではないかもしれません。」

# 1. 文分割
sentences = [
    "知識とは何だろうか",
    "しかし、それは本当の答えではないかもしれません"
]

# 2. 各文でパターンを検出
- "だろうか" を検出 → 注意度1.0
- "しかし" を検出 → 注意度0.9
- "、" を検出 → 注意度0.95
- "。" を検出 → 注意度0.95

# 3. 注意マスクの生成
[0.8, 0.8, 0.8, 1.0, 1.0, 0.95, 0.9, 0.8, 0.8, 0.95, ...]
```

5. **目的**:
- ソクラテス式問答の重要な要素に注目
  - 問いかけの部分（「だろうか」など）
  - 論理の展開（接続詞）
  - 文の区切り（句読点）
- これにより、モデルが:
  - 適切な問いかけ方を学習
  - 論理的な文章の構造を理解
  - 文の区切りを適切に認識

このように、ソクラテス式問答の特徴的な表現や構造に対して、モデルがより注意を払うように設定しています。

        
</div>
    </details>
</div>





<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>


<div style="background-color: #e6f3ff; padding: 10px; border-left: 5px solid #2196F3; margin: 10px; width: 95%;">
<summary style="color: #2196F3; font-size: 1.17em; font-weight: bold;">Q. 質問内容</summary>
<p style="color: #2196F3;">これは目的関数に関わることですか？それともあくまで評価時の項目ですか</p>
</div>


<div style="background-color: #f5f5f5; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #4CAF50; font-size: 1.17em; font-weight: bold;">A. 解答</summary>
        <div style="color: #4CAF50;">







このコードは**目的関数（損失関数）に直接影響を与えます**。具体的には、Attention Maskを通じてモデルの学習過程自体に影響を与えます。

以下で詳しく説明します：

1. **Attention Maskの役割**:
```python
# 通常のAttention Mask（0か1のみ）
normal_mask = [1, 1, 1, 1, 0, 0]  # 1: 有効なトークン、0: パディング

# このコードによるAttention Mask（0.8～1.0の値）
custom_mask = [0.8, 1.0, 0.9, 0.95, 0, 0]
# 0.8: 基本的な注意度
# 1.0: 重要な文末表現（「だろうか」など）
# 0.9: 接続詞
# 0.95: 句読点
```

2. **学習への影響**:
```python
# 例：ソクラテス式の問答
text = "知識とは何だろうか。しかし、それは表面的な理解かもしれない。"

# 注意度の重み付け
"知識とは"        -> 0.8  # 基本の注意度
"何だろうか"      -> 1.0  # 問いかけ表現として重要
"しかし"          -> 0.9  # 論理展開の接続詞
"、"              -> 0.95 # 文の区切り
"かもしれない"    -> 1.0  # 思考を促す表現
```

3. **目的関数への影響**:
- モデルの学習時に、これらの重み付けが損失関数の計算に影響
- より重要な部分（注意度が高い部分）の誤差がより大きく反映される
- 結果として、モデルは:
  - 問いかけ表現の適切な使用
  - 論理的な文章の構造
  - 適切な句読点の使用
  を重点的に学習する

4. **評価時との違い**:
```python
# 学習時：
- Attention Maskの重みが損失関数に直接影響
- モデルのパラメータ更新に影響

# 評価時：
- compute_metrics()関数で別途評価
- モデルの出力を評価するだけで、学習には影響しない
```

したがって、このコードは：
- 単なる評価指標ではなく
- 学習プロセス自体に組み込まれた
- モデルの振る舞いを直接制御する仕組み

として機能しています。これにより、ソクラテス式問答の特徴をモデルにより効果的に学習させることができます。

        
</div>
    </details>
</div>


In [ ]:




# Optimize dataset processing
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=16,
    num_proc=2,
    load_from_cache_file=True,
    desc="Tokenizing datasets",
    remove_columns=dataset.column_names,
)




<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">









このコードは、データセットのトークン化処理を最適化して実行するものです。具体的に説明します：

```python
tokenized_dataset = dataset.map(
    tokenize_function,     # 先ほど定義したトークン化関数
    batched=True,         # バッチ処理を有効化
    batch_size=16,        # 一度に処理するサンプル数
    num_proc=2,           # 使用するプロセス数
    load_from_cache_file=True,  # キャッシュの使用
    desc="Tokenizing datasets",  # 進捗表示の説明
    remove_columns=dataset.column_names,  # 元のカラムを削除
)
```


**具体例で説明**:

1. **バッチ処理**:
```python
# バッチ処理なし（非効率）
for text in dataset:
    tokenize_function(text)  # 1つずつ処理

# バッチ処理あり（効率的）
batch = [
    "User: 知識とは何でしょうか\nAssistant: 考えてみましょう...",
    "User: 真理について\nAssistant: 興味深い質問ですね...",
    ...  # 16個のテキストを一度に処理
]
tokenize_function(batch)
```


2. **並列処理**:
```python
num_proc=2  # 2つのプロセスで並列処理

プロセス1: [テキスト1, テキスト2, テキスト3, ...]
プロセス2: [テキスト16, テキスト17, テキスト18, ...]
```


3. **キャッシュの使用**:
```python
load_from_cache_file=True

# 初回実行時
1. トークン化を実行
2. 結果をキャッシュに保存

# 2回目以降の実行時
1. キャッシュがあれば、それを読み込む
2. トークン化処理をスキップ
```


4. **カラムの削除**:
```python
# 処理前のデータセット
{
    'text': ['対話テキスト1', '対話テキスト2', ...],
    'other_column': [...]
}

# 処理後のデータセット（remove_columns効果）
{
    'input_ids': [[101, 2345, ...], [101, 5678, ...], ...],
    'attention_mask': [[1, 1, ...], [1, 1, ...], ...]
}
```


**最適化の利点**:

1. **処理速度の向上**:
   - バッチ処理による効率化
   - 複数プロセスによる並列処理
   - キャッシュによる再計算の回避

2. **メモリ効率**:
   - バッチ単位での処理で、メモリ使用を制御
   - 不要なカラムの削除

3. **進捗の可視化**:
   - `desc`パラメータによる処理状況の表示
   - 長時間の処理の進行状況が分かる

このように、大量の対話データを効率的にトークン化処理するための最適化が施されています。

        
</div>
    </details>
</div>



# Add dataset validation
def validate_dataset(dataset):
    # Check first element
    first_item = dataset[0]
    print("Validated first item structure:")
    print(f"Keys: {first_item.keys()}")
    print(f"input_ids type: {type(first_item['input_ids'])}")
    print(f"input_ids length: {len(first_item['input_ids'])}")
    return dataset








<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">









このコードはデータセットの構造を検証する関数です。具体的な例を使って説明します：

```python
def validate_dataset(dataset):
    # 最初の要素を取得して検証
    first_item = dataset[0]
    
    # 構造の確認と出力
    print("Validated first item structure:")
    print(f"Keys: {first_item.keys()}")  # データの持つキーを表示
    print(f"input_ids type: {type(first_item['input_ids'])}")  # トークンIDの型を確認
    print(f"input_ids length: {len(first_item['input_ids'])}")  # トークン列の長さを確認
    return dataset
```


**想定される出力例**:
```python
Validated first item structure:
Keys: dict_keys(['input_ids', 'attention_mask'])
input_ids type: <class 'list'>
input_ids length: 512
```


**具体的な検証内容**:

1. **キーの確認**:
```python
# 期待される構造
first_item = {
    'input_ids': [...],      # トークンのID列
    'attention_mask': [...]  # 注意マスク
}

# キーの出力
print(f"Keys: {first_item.keys()}")
# 期待される出力: Keys: dict_keys(['input_ids', 'attention_mask'])
```


2. **データ型の確認**:
```python
# input_idsの型チェック
print(f"input_ids type: {type(first_item['input_ids'])}")
# 期待される出力: input_ids type: <class 'list'>

# 実際のデータ例（ソクラテス式問答の場合）
input_ids = [
    101,  # [BOS]トークン
    2345, # "User:"に相当するトークン
    6789, # "知識とは何でしょうか"のトークン
    102,  # [EOS]トークン
    ...
]
```


3. **長さの確認**:
```python
# トークン列の長さチェック
print(f"input_ids length: {len(first_item['input_ids'])}")
# 期待される出力: input_ids length: 512

# 長さが512である理由：
# - TOKENIZE_MAX_LENGTH = 512 に設定されている
# - 短い文章はパディング、長い文章は切り詰めで512に統一
```


**この検証が重要な理由**:

1. **データ形式の確認**:
   - モデルが期待する入力形式になっているか
   - 必要なキー（input_ids, attention_mask）が存在するか

2. **トークン化の確認**:
   - トークン化が正しく行われているか
   - 設定した最大長（512）が守られているか

3. **早期エラー検出**:
   - トレーニング開始前に問題を発見できる
   - データ形式の不整合を防ぐ

このバリデーション関数は、トレーニングの前にデータセットが正しい形式になっていることを確認する重要なステップです。

        
</div>
    </details>
</div>


In [ ]:
tokenized_dataset = validate_dataset(tokenized_dataset)


tokenized_dataset = tokenized_dataset.map(
    preprocess_function,
    batched=True,
    desc="Applying attention masking"
)


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">











このコードは、トークン化されたデータセットの検証と前処理を順番に行っています：

1. **データセット検証**:
```python
# トークン化されたデータセットを検証
tokenized_dataset = validate_dataset(tokenized_dataset)

# 想定される出力例
Validated first item structure:
Keys: dict_keys(['input_ids', 'attention_mask'])
input_ids type: <class 'list'>
input_ids length: 512
```

2. **注意マスクの適用**:
```python
# 検証済みデータセットに前処理（注意マスク）を適用
tokenized_dataset = tokenized_dataset.map(
    preprocess_function,  # 先ほど説明した前処理関数
    batched=True,        # バッチ処理を有効化
    desc="Applying attention masking"  # 進捗表示の説明
)
```

**処理の流れ**:
```python
# 1. 元のデータ（data/dialogue/processed/kaggle_model.json）
{
    "messages": [
        {"role": "user", "content": "知識とは何でしょうか"},
        {"role": "model", "content": "興味深い質問ですね。考えてみましょう..."}
    ]
}

# 2. トークン化後のデータ
{
    'input_ids': [101, 2345, 6789, ...],  # トークンID列
    'attention_mask': [1, 1, 1, ...]      # 基本的な注意マスク
}

# 3. 検証（validate_dataset）
- キーの存在確認
- データ型の確認
- 長さの確認

# 4. 注意マスクの適用（preprocess_function）
{
    'input_ids': [101, 2345, 6789, ...],
    'attention_mask': [0.8, 1.0, 0.9, ...]  # カスタマイズされた注意マスク
}
```

**この順序が重要な理由**:

1. **安全性**:
   - 前処理を適用する前にデータ構造を検証
   - 問題があれば早期に発見可能

2. **デバッグ容易性**:
   - 各ステップの出力を確認可能
   - エラーの原因特定が容易

3. **処理の一貫性**:
   - 検証済みのデータに対して前処理を適用
   - データの品質を保証

このように、データの検証と前処理を順序立てて行うことで、安全で信頼性の高いデータセットの準備が可能になります。

        
</div>
    </details>
</div>
